In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import cv2
# Replace 'path/to/dataset' with the actual path to your dataset folder
dataset_path = '/content/drive/MyDrive/DL&SIP/ILLUMINATION'

# Get a list of all the folder names in the dataset directory
class_names = os.listdir(dataset_path)
print(class_names)


['corkscrew', 'bowl', 'bottle', 'cup', 'cottonswab', 'cushion', 'knife']


In [3]:
import os
import cv2
# Replace 'path/to/dataset' with the actual path to your dataset folder
dataset_path_1 = '/content/drive/MyDrive/DL&SIP/occlusion'

# Get a list of all the folder names in the dataset directory
class_names_1 = os.listdir(dataset_path_1)
print(class_names_1)

['cushion', 'bottle', 'knife', 'cottonswab', 'cup', 'corkscrew', 'bowl']


In [4]:
img_list=[]
label_list=[]
target_size = (170, 170)

for class_name in class_names:
    class_path = os.path.join(dataset_path, class_name)
    for filename in os.listdir(class_path):
        img_path = os.path.join(class_path, filename)
        img=cv2.imread(img_path)
        img = cv2.resize(img, target_size)
        img_list.append(img)
        label_list.append(class_name)

In [5]:
for class_name in class_names_1:
    class_path = os.path.join(dataset_path_1, class_name)
    for filename in os.listdir(class_path):
        img_path = os.path.join(class_path, filename)
        img=cv2.imread(img_path)
        img = cv2.resize(img, target_size)
        img_list.append(img)
        label_list.append(class_name)

In [6]:
import numpy as np
img_array = np.array(img_list)

In [7]:
def Normalize_Images(images):
 # convert from integers to floats
 img_norm = images.astype('float32')
 # normalize to range 0-1
 img_norm = img_norm / 255.0
 # return normalized images
 return img_norm

In [8]:
 img_norm=Normalize_Images(img_array)
 img_norm

array([[[[0.4862745 , 0.5137255 , 0.42745098],
         [0.6156863 , 0.6431373 , 0.53333336],
         [0.62352943, 0.6431373 , 0.5254902 ],
         ...,
         [0.30588236, 0.3647059 , 0.25882354],
         [0.2627451 , 0.33333334, 0.23137255],
         [0.27058825, 0.34117648, 0.2509804 ]],

        [[0.62352943, 0.6509804 , 0.54509807],
         [0.6156863 , 0.6431373 , 0.53333336],
         [0.6156863 , 0.6392157 , 0.5176471 ],
         ...,
         [0.3372549 , 0.39607844, 0.2784314 ],
         [0.30588236, 0.36862746, 0.25882354],
         [0.28627452, 0.3529412 , 0.2509804 ]],

        [[0.627451  , 0.654902  , 0.54901963],
         [0.62352943, 0.6509804 , 0.5411765 ],
         [0.6313726 , 0.654902  , 0.53333336],
         ...,
         [0.45882353, 0.5176471 , 0.39215687],
         [0.3882353 , 0.44705883, 0.3254902 ],
         [0.32941177, 0.39215687, 0.28235295]],

        ...,

        [[0.22352941, 0.49411765, 0.62352943],
         [0.21960784, 0.49019608, 0.61960787]

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label_array = le.fit_transform(label_list)

In [10]:
from keras.utils import to_categorical
label_array_cat = to_categorical(label_array)
label_array_cat

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
img_norm.shape

(5601, 170, 170, 3)

In [12]:
label_array_cat.shape

(5601, 7)

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.callbacks import EarlyStopping

def cnn_model(regularization_type='dropout', regularization_rate=0.001):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(170, 170, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())

    if regularization_type == 'l1':
    	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l1(regularization_rate)))
    	model.add(Dense(64, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l1(regularization_rate)))
    	model.add(Dense(32, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l1(regularization_rate)))
   
    elif regularization_type == 'l2':
    	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l1(regularization_rate)))
    	model.add(Dense(64, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l1(regularization_rate)))
    	model.add(Dense(32, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l1(regularization_rate)))

    elif regularization_type == 'l1+dropout':
    	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l1(regularization_rate)))
    	model.add(Dropout(0.2))
    	model.add(Dense(64, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l1(regularization_rate)))
    	model.add(Dropout(0.2))
    	model.add(Dense(32, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l1(regularization_rate)))
    	model.add(Dropout(0.4))

    elif regularization_type == 'l2+dropout':
    	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(regularization_rate)))
    	model.add(Dropout(0.2))
    	model.add(Dense(64, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(regularization_rate)))
    	model.add(Dropout(0.2))
    	model.add(Dense(32, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(regularization_rate)))
    	model.add(Dropout(0.4))

    else:
    	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    	model.add(Dropout(0.2))
    	model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    	model.add(Dropout(0.2))
    	model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    	model.add(Dropout(0.4))

    model.add(Dense(7, activation='softmax'))
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# Example usage with L1 regularization
model_l1 = cnn_model(regularization_type='l2', regularization_rate=0.001)

# Example usage with L2 regularization
model_l2 = cnn_model(regularization_type='l1', regularization_rate=0.001)

# Example usage with L1 regularization and different regularization rate and dropout rate
model_l1_dropout = cnn_model(regularization_type='l1+dropout', regularization_rate=0.001)

# Example usage with L2 regularization and different regularization rate and dropout rate
model_l2_dropout = cnn_model(regularization_type='l2+dropout', regularization_rate=0.001)

# Example usage with dropout
model_dropout = cnn_model()

In [14]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(img_norm, label_array_cat, test_size=0.3, shuffle=True)

# Split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)


In [17]:
history = model_l1.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=0)

In [18]:
_, acc = model_l1.evaluate(X_test, y_test, verbose=0)
print('> %.3f' % (acc * 100.0))

> 87.983


In [20]:
history = model_l2.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=0)

In [ ]:
_, acc = model_l2.evaluate(X_test, y_test, verbose=0)
print('> %.3f' % (acc * 100.0))

In [18]:
history = model_l1_dropout.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=0)

In [19]:
_, acc = model_l1_dropout.evaluate(X_test, y_test, verbose=0)
print('> %.3f' % (acc * 100.0))

> 79.655


In [15]:
history = model_dropout.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=0)

In [16]:
_, acc = model_dropout.evaluate(X_test, y_test, verbose=0)
print('> %.3f' % (acc * 100.0))

> 92.326


In [20]:
history = model_l2_dropout.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=0)

In [21]:
_, acc = model_l2_dropout.evaluate(X_test, y_test, verbose=0)
print('> %.3f' % (acc * 100.0))

> 95.360
